In [14]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

# Load the data
data_path = ''
df = pd.read_excel(data_path, index_col=0)

# Function to calculate logarithmic returns
def calculate_returns(df, weeks):
    return np.log(df / df.shift(weeks))

# Function to execute the momentum strategy with overlapping periods
def momentum_strategy_overlapping(df, observation_weeks, holding_weeks):
    weekly_returns = calculate_returns(df, 1)
    
    winner_returns_list = []
    winner_minus_loser_returns_list = []
    combo_returns_list = []

    total_periods = len(df) - observation_weeks - holding_weeks + 1 
    
    for period_start in range(total_periods):
        observation_period = weekly_returns.iloc[period_start:period_start + observation_weeks]
        mean_observation_returns = observation_period.mean()
        
        winners = mean_observation_returns.nlargest(8).index
        losers = mean_observation_returns.nsmallest(8).index
        
        holding_period = weekly_returns.iloc[period_start + observation_weeks:period_start + observation_weeks + holding_weeks]
        
        winner_returns = holding_period[winners].mean(axis=1)
        loser_returns = holding_period[losers].mean(axis=1)
        market_returns = holding_period['上证综指'].mean()
        
        winner_excess_returns = winner_returns - market_returns
        winner_minus_loser_returns = winner_returns - loser_returns
        combo_returns = winner_returns - loser_returns - market_returns
        
        winner_returns_list.extend(winner_excess_returns)
        winner_minus_loser_returns_list.extend(winner_minus_loser_returns)
        combo_returns_list.extend(combo_returns)

    winner_over_market_pvalue = ttest_1samp(winner_returns_list, 0).pvalue
    winner_minus_loser_pvalue = ttest_1samp(winner_minus_loser_returns_list, 0).pvalue
    combo_pvalue = ttest_1samp(combo_returns_list, 0).pvalue

    return (winner_over_market_pvalue, np.mean(winner_returns_list),
            winner_minus_loser_pvalue, np.mean(winner_minus_loser_returns_list),
            combo_pvalue, np.mean(combo_returns_list))

observation_periods = [1, 2,3,4, 6,8,12,16]
holding_periods = [1, 2,3,4,6,8,12,16]

returns_df = pd.DataFrame(index=holding_periods, columns=observation_periods)
pvalues_df = pd.DataFrame(index=holding_periods, columns=observation_periods)

for observation_period in observation_periods:
    for holding_period in holding_periods:
        (winner_over_market_pvalue, mean_winner_returns,
         winner_minus_loser_pvalue, mean_winner_minus_loser_returns,
         combo_pvalue, mean_combo_returns) = momentum_strategy_overlapping(df, observation_period, holding_period)
        
        returns_df.loc[holding_period, observation_period] = mean_combo_returns
        pvalues_df.loc[holding_period, observation_period] = combo_pvalue

returns_output_path = 'returns_output.xlsx'
pvalues_output_path = 'pvalues_output.xlsx'

returns_df.to_excel(returns_output_path)
pvalues_df.to_excel(pvalues_output_path)

print("Returns saved to:", returns_output_path)
print("P-values saved to:", pvalues_output_path)


Returns saved to: returns_output.xlsx
P-values saved to: pvalues_output.xlsx


In [21]:
# 初始化一个空的DataFrame来存储结果
results_df = pd.DataFrame()

# 遍历所有的观察期和持有期组合来运行动量策略，并保存p值和收益率
for observation in observation_periods:
    for holding in holding_periods:
        # 调用动量策略函数，传入当前的观察期和持有期
        # 函数返回赢家组合超过市场的p值、平均收益率；
        # 赢家组合减去输家组合的p值、平均收益率；
        # 以及组合的p值和平均收益率
        p_value_winner_market, avg_return_winner, \
        p_value_winner_loser, avg_return_winner_loser,\
        p_value_combo, avg_return_combo= momentum_strategy_overlapping(df, observation, holding)
        
        # 为当前迭代结果创建一个DataFrame
        current_results = pd.DataFrame({
            'Observation Period': [observation],  # 观察期
            'Holding Period': [holding], # 持有期
            'Winner over Market p-value': [p_value_winner_market], # 赢家超过市场的p值
            'Winner over Market Return': [avg_return_winner],  # 赢家超过市场的平均收益率
            'Winner minus Loser p-value': [p_value_winner_loser], # 赢家减去输家的p值
            'Winner minus Loser Return': [avg_return_winner_loser], # 赢家减去输家的平均收益率
            'combo p-value': [p_value_combo], # 组合的p值
            'combo Return': [avg_return_combo] # 组合的平均收益率
        })

        # 将当前结果DataFrame与主结果DataFrame连接起来
        # ignore_index=True参数意味着不保留连接DataFrame的原始索引
        results_df = pd.concat([results_df, current_results], ignore_index=True)

# 显示结果
print(results_df)


    Observation Period  Holding Period  Winner over Market p-value  \
0                    1               1                    0.038154   
1                    1               2                    0.039462   
2                    1               3                    0.021288   
3                    1               4                    0.018059   
4                    1               6                    0.031742   
5                    1               8                    0.018591   
6                    1              10                    0.012994   
7                    2               1                    0.001160   
8                    2               2                    0.007118   
9                    2               3                    0.007196   
10                   2               4                    0.012211   
11                   2               6                    0.020825   
12                   2               8                    0.011087   
13                  

In [22]:
import matplotlib.pyplot as plt

# 分别提取不同策略的p值
winner_market_p_values = results_df['Winner over Market p-value'] # 提取“胜者相对市场”的p值
winner_loser_p_values = results_df['Winner minus Loser p-value'] # 提取“胜者减输家”的p值
combo_p_values = results_df['combo p-value'] # 提取“组合”的p值

# 设置图表大小
plt.figure(figsize=(15, 5)) # 定义一个图形界面的大小为15x5英寸

# 绘制“胜者相对市场”的p值条形图
plt.subplot(1, 3, 1) # 在1行3列的子图布局中的第1列位置创建一个子图
plt.bar(range(len(winner_market_p_values)), winner_market_p_values, color='blue') # 使用蓝色条形图表示“胜者相对市场”的p值
plt.xticks(range(len(winner_market_p_values)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45)
plt.axhline(y=0.05, color='red', linestyle='--')  # 添加y=0.05的红色虚线，标识统计显著性阈值
plt.title('Winner over Market p-values') # 设置图标题
plt.ylabel('p-value') # 设置y轴标签为“p-value”

# 绘制“胜者减输家”的p值条形图
plt.subplot(1, 3, 2) # 在1行3列的子图布局中的第2列位置创建一个子图
plt.bar(range(len(winner_loser_p_values)), winner_loser_p_values, color='purple') # 使用紫色条形图表示“胜者减输家”的p值
plt.xticks(range(len(winner_loser_p_values)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45)
plt.axhline(y=0.05, color='red', linestyle='--') # 添加统计显著性阈值线
plt.title('Winner minus Loser p-values') # 设置图标题

# 绘制“组合”的p值条形图
plt.subplot(1, 3, 3) # 在1行3列的子图布局中的第3列位置创建一个子图
plt.bar(range(len(combo_p_values)), combo_p_values, color='green') # 使用绿色条形图表示“组合”的p值
plt.xticks(range(len(combo_p_values)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45)
plt.axhline(y=0.05, color='red', linestyle='--') # 添加统计显著性阈值线
plt.title('combo p-values') # 设置图标题

# 调整布局并显示图表
plt.tight_layout() # 调整子图布局，确保子图之间的空间和布局整齐
plt.show() # 显示图表


ImportError: cannot import name 'backend_agg' from 'matplotlib.backends' (c:\Users\20517\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\backends\__init__.py)

In [23]:
# 分别提取不同策略的平均收益率
winner_market_returns = results_df['Winner over Market Return'] # 提取“胜者相对市场”的平均收益率
winner_loser_returns = results_df['Winner minus Loser Return'] # 提取“胜者减输家”的平均收益率
combo_returns = results_df['combo Return'] # 提取“组合”的平均收益率

# 设置图表大小
plt.figure(figsize=(15, 5)) # 定义图形界面的大小为15x5英寸

# 绘制“胜者相对市场”的平均收益率条形图
plt.subplot(1, 3, 1) # 在1行3列的子图布局中的第1列位置创建一个子图
plt.bar(range(len(winner_market_returns)), winner_market_returns, color='blue') # 使用蓝色条形图表示“胜者相对市场”的平均收益率
plt.xticks(range(len(winner_market_returns)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45) # 设置x轴刻度标签，显示观察期-持有期，并旋转45度以改善可读性
plt.title('Winner over Market Returns') # 设置图标题
plt.ylabel('Average Return') # 设置y轴标签为“Average Return”

# 绘制“胜者减输家”的平均收益率条形图
plt.subplot(1, 3, 2) # 在1行3列的子图布局中的第2列位置创建一个子图
plt.bar(range(len(winner_loser_returns)), winner_loser_returns, color='purple') # 使用紫色条形图表示“胜者减输家”的平均收益率
plt.xticks(range(len(winner_loser_returns)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45) # 设置x轴刻度标签
plt.title('Winner minus Loser Returns') # 设置图标题

# 绘制“组合”的平均收益率条形图
plt.subplot(1, 3, 3) # 在1行3列的子图布局中的第3列位置创建一个子图
plt.bar(range(len(combo_returns)), combo_returns, color='green') # 使用绿色条形图表示“组合”的平均收益率
plt.xticks(range(len(combo_returns)), [f'{op}-{hp}' for op, hp in zip(results_df['Observation Period'], results_df['Holding Period'])], rotation=45)
plt.title('combo Returns') # 设置图标题

plt.tight_layout() # 调整子图布局，确保子图之间的空间和布局整齐
plt.show() # 显示图表



ImportError: cannot import name 'backend_agg' from 'matplotlib.backends' (c:\Users\20517\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\backends\__init__.py)